In [1]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from imutils import paths

import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

2022-04-18 15:13:00.032010: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-18 15:13:00.032148: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class modelNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        
        model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

In [3]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32

In [4]:
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [5]:
imagePaths = sorted(list(paths.list_images("augData")))
random.seed(42)
random.shuffle(imagePaths)

In [155]:
print(len(imagePaths))

9667


In [6]:
for imagePath in imagePaths:    
    label = imagePath.split(os.path.sep)[-2]
    
    if label == "triple_rider":
        label = 1
        labels.append(label)
    else:
        label = 0
        labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    

In [7]:
unique, counts = np.unique(np.array(labels), return_counts=True)
dict(zip(unique, counts))

{0: 5015, 1: 4652}

In [134]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [135]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [136]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	fill_mode="nearest")

In [137]:
print("[INFO] compiling model...")
model = modelNet.build(width=28, height=28, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [138]:
print("[INFO] training network...")
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)
# save the model to disk
print("[INFO] serializing network...")
model.save("model_1.h5", save_format="h5")

[INFO] training network...
Epoch 1/25
3/3 [==============================] - 6s 657ms/step - loss: 0.7850 - accuracy: 0.4583 - val_loss: 0.7087 - val_accuracy: 0.5312
Epoch 2/25
3/3 [==============================] - 1s 330ms/step - loss: 0.7280 - accuracy: 0.4167 - val_loss: 0.7178 - val_accuracy: 0.4688
Epoch 3/25
3/3 [==============================] - 1s 275ms/step - loss: 0.6909 - accuracy: 0.5000 - val_loss: 0.6949 - val_accuracy: 0.4688
Epoch 4/25
3/3 [==============================] - 1s 281ms/step - loss: 0.6814 - accuracy: 0.6042 - val_loss: 0.6933 - val_accuracy: 0.5625
Epoch 5/25
3/3 [==============================] - 1s 305ms/step - loss: 0.6757 - accuracy: 0.7917 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 6/25
3/3 [==============================] - 1s 288ms/step - loss: 0.6621 - accuracy: 0.8125 - val_loss: 0.6955 - val_accuracy: 0.5312
Epoch 7/25
3/3 [==============================] - 1s 276ms/step - loss: 0.6378 - accuracy: 0.8229 - val_loss: 0.7320 - val_accuracy: 

In [139]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Santa/Not Santa")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [140]:
predictions = model.predict(x=testX, batch_size=32)

print(predictions.shape)

(32, 2)


In [141]:
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=["triple_rider","double rider"]))

              precision    recall  f1-score   support

triple_rider       0.67      0.47      0.55        17
double rider       0.55      0.73      0.63        15

    accuracy                           0.59        32
   macro avg       0.61      0.60      0.59        32
weighted avg       0.61      0.59      0.59        32

